# Importing relevant packages

In [1]:
import pandas as pd
import numpy as np
import warnings

pd.set_option("mode.copy_on_write", True)

# Reading and Loading (Takes 10 minutes to get relevant states)

In [2]:
# reading a small sample
all = pd.read_csv(
    "~/Desktop/arcos_all_washpost.zip",
    nrows=200,
    delimiter="\t",
    compression="zip",
)
all.head(5)

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
0,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-02-08,"Par Pharmaceutical, Inc.",McKesson Corporation,0.89650,100.0,1344.75
1,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,7.5,2011-03-07,SpecGx LLC,McKesson Corporation,0.45405,100.0,454.05
2,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-03-10,"Par Pharmaceutical, Inc.",McKesson Corporation,3.58600,400.0,5379.00
3,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-05,SpecGx LLC,McKesson Corporation,1.51350,500.0,1513.50
4,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-06,SpecGx LLC,McKesson Corporation,0.30270,100.0,302.70


In [4]:
# Columns to keep
variables_to_keep = [
    "BUYER_STATE",
    "BUYER_COUNTY",
    "DRUG_NAME",
    "TRANSACTION_DATE",
    "CALC_BASE_WT_IN_GM",
    "MME_Conversion_Factor",
]

# Chunking
chunk_size = 100000
chunk_list = []
states_to_keep = ["FL", "WA", "OR", "GA", "OK", "AL", "CO", "MN"]


# Read in chunks, filtering for Texas and relevant columns
for chunk in pd.read_csv(
    "~/Desktop/arcos_all_washpost.zip",
    delimiter="\t",
    chunksize=chunk_size,
    compression="zip",
    usecols=variables_to_keep,
):
    filtered_chunk = chunk[chunk["BUYER_STATE"].isin(states_to_keep)]
    chunk_list.append(filtered_chunk)
df = pd.concat(chunk_list, ignore_index=True)
df

,BUYER_STATE,BUYER_COUNTY,DRUG_NAME,MME_Conversion_Factor,TRANSACTION_DATE,CALC_BASE_WT_IN_GM
0,GA,RICHMOND,OXYCODONE,1.5,2019-06-19,1.344750
1,GA,COWETA,OXYCODONE,1.5,2019-06-21,3.361875
2,GA,CLARKE,HYDROCODONE,1.0,2019-06-21,2.270250
3,GA,WHITFIELD,HYDROCODONE,1.0,2019-06-21,2.270250
4,GA,COBB,HYDROCODONE,1.0,2019-06-21,2.270250
...,...,...,...,...,...,...
69654434,OR,MULTNOMAH,OXYCODONE,1.5,2010-12-13,5.379000
69654435,OR,MULTNOMAH,OXYCODONE,1.5,2011-01-03,29.584500
69654436,OR,MULTNOMAH,OXYCODONE,1.5,2011-02-15,8.068500
69654437,OR,MULTNOMAH,HYDROCODONE,1.0,2011-02-18,7.567500


# Calculating relevant data

In [ ]:
# dropping NAs
all = df[
    df["BUYER_STATE"].isin(["FL", "WA", "OR", "GA", "OK", "AL", "CO", "MN"])
].dropna()
all.head(5)

,BUYER_STATE,BUYER_COUNTY,DRUG_NAME,MME_Conversion_Factor,TRANSACTION_DATE,CALC_BASE_WT_IN_GM
0,GA,RICHMOND,OXYCODONE,1.5,2019-06-19,1.344750
1,GA,COWETA,OXYCODONE,1.5,2019-06-21,3.361875
2,GA,CLARKE,HYDROCODONE,1.0,2019-06-21,2.270250
3,GA,WHITFIELD,HYDROCODONE,1.0,2019-06-21,2.270250
4,GA,COBB,HYDROCODONE,1.0,2019-06-21,2.270250


In [31]:
# Getting the year
all["date"] = pd.to_datetime(
    all["TRANSACTION_DATE"], format="%Y-%m-%d", errors="coerce"
)
all["year"] = all["date"].dt.year

# Make an estimate of total morphine equivalent shipments
all["MME_Conversion_Factor"] = pd.to_numeric(
    all["MME_Conversion_Factor"], errors="coerce"
)
all["morphine_equivalent_g"] = all["CALC_BASE_WT_IN_GM"] * all["MME_Conversion_Factor"]
# Group by year, state, and county, and sum the morphine equivalent shipments
df_grouped = (
    all.groupby(["year", "BUYER_STATE", "BUYER_COUNTY"])
    .morphine_equivalent_g.sum()
    .reset_index()
)
df_grouped

,year,BUYER_STATE,BUYER_COUNTY,morphine_equivalent_g
0,2006,AL,AUTAUGA,17100.542225
1,2006,AL,BALDWIN,51039.829050
2,2006,AL,BARBOUR,5764.019122
3,2006,AL,BIBB,6278.096539
4,2006,AL,BLOUNT,10418.423330
...,...,...,...,...
8117,2019,WA,WAHKIAKUM,932.537438
8118,2019,WA,WALLA WALLA,13420.082768
8119,2019,WA,WHATCOM,40160.700638
8120,2019,WA,WHITMAN,7147.307865


In [39]:
# check that the relevant states are in the dataset
assert len(df_grouped["BUYER_STATE"].unique()) == 8
df_grouped.head(5)

,year,BUYER_STATE,BUYER_COUNTY,morphine_equivalent_g
0,2006,AL,AUTAUGA,17100.542225
1,2006,AL,BALDWIN,51039.829050
2,2006,AL,BARBOUR,5764.019122
3,2006,AL,BIBB,6278.096539
4,2006,AL,BLOUNT,10418.423330


# Checking Florida & Constants GA, AL, OK

In [ ]:
florida_and_constants = all[all["BUYER_STATE"].isin(["FL", "AL", "GA", "OK"])]
# florida has 67 counties
assert (
    len(
        florida_and_constants[florida_and_constants["BUYER_STATE"] == "FL"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 67
)
# georgia has 159 counties
assert (
    len(
        florida_and_constants[florida_and_constants["BUYER_STATE"] == "GA"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 159
)
# oklahoma has 77 counties
assert (
    len(
        florida_and_constants[florida_and_constants["BUYER_STATE"] == "OK"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 77
)
# alabama has 67 counties
assert (
    len(
        florida_and_constants[florida_and_constants["BUYER_STATE"] == "AL"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 67
)
print(florida_and_constants["year"].unique())
florida_and_constants.head(5)

[2019 2015 2016 2017 2018 2009 2010 2011 2012 2013 2014 2007 2008 2006]


,BUYER_STATE,BUYER_COUNTY,DRUG_NAME,MME_Conversion_Factor,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,date,year,morphine_equivalent_g
0,GA,RICHMOND,OXYCODONE,1.5,2019-06-19,1.344750,2019-06-19,2019,2.017125
1,GA,COWETA,OXYCODONE,1.5,2019-06-21,3.361875,2019-06-21,2019,5.042813
2,GA,CLARKE,HYDROCODONE,1.0,2019-06-21,2.270250,2019-06-21,2019,2.270250
3,GA,WHITFIELD,HYDROCODONE,1.0,2019-06-21,2.270250,2019-06-21,2019,2.270250
4,GA,COBB,HYDROCODONE,1.0,2019-06-21,2.270250,2019-06-21,2019,2.270250


# Checking Washington & Constants CO, OR, MN

In [38]:
washington_and_constants = all[all["BUYER_STATE"].isin(["WA", "CO", "OR", "MN"])]
# washington has 39 counties
assert (
    len(
        washington_and_constants[washington_and_constants["BUYER_STATE"] == "WA"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 39
)
# colorado has 64 counties
assert (
    len(
        washington_and_constants[washington_and_constants["BUYER_STATE"] == "CO"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 64
)
# oregon has 36 counties
assert (
    len(
        washington_and_constants[washington_and_constants["BUYER_STATE"] == "OR"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 36
)
# minnesota has 87 counties
assert (
    len(
        washington_and_constants[washington_and_constants["BUYER_STATE"] == "MN"][
            "BUYER_COUNTY"
        ].unique()
    )
    <= 87
)
print(washington_and_constants["year"].unique())
washington_and_constants.head(5)

[2016 2017 2018 2019 2015 2011 2012 2013 2014 2006 2007 2008 2009 2010]


,BUYER_STATE,BUYER_COUNTY,DRUG_NAME,MME_Conversion_Factor,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,date,year,morphine_equivalent_g
10855717,MN,HENNEPIN,OXYCODONE,1.5,2016-02-05,5.37900,2016-02-05,2016,8.068500
10855718,MN,DAKOTA,OXYCODONE,1.5,2016-02-05,0.44825,2016-02-05,2016,0.672375
10855719,MN,HENNEPIN,OXYCODONE,1.5,2016-02-05,0.44825,2016-02-05,2016,0.672375
10855720,MN,HENNEPIN,OXYCODONE,1.5,2016-02-05,2.24125,2016-02-05,2016,3.361875
10855721,MN,ANOKA,OXYCODONE,1.5,2016-02-05,3.58600,2016-02-05,2016,5.379000


# SAVE TO PARQUET

In [ ]:
# save to parquet
df_grouped.to_parquet("data/opioid_shipment_WA_FL_andconstants.parquet", index=False)